# 1. Setup

In [1]:
%pip install transformers

# 2. Load BERT model to embeddings

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load the BERT model
model_name = 'bert-base-uncased'  # DMetaSoul/sbert-chinese-general-v2
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to compute sentence embeddings
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    embeddings = embeddings.detach().numpy()
    return embeddings[0]  # return a 2-dimensional embedding


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

# 3. 讀取資料

In [ ]:
import json

with open('eng_sentences_cut100_1.json') as f:
    eng_data = json.load(f)

with open('eng_sentences_cut900_1.json') as f:
    paired_data = json.load(f)
# for item in eng_data:
#       print(list(item.keys())[0])

# 用英文原句做Embedding

In [ ]:
k = 10
retrieve_output = []

for eng in eng_data:
    datastore_embeddings =[]
    sentence = list(eng.keys())[0]
    print(f"query: {sentence}")
    retrieve_output.append([list(eng.keys())[0],list(eng.values())[0]])
    query_embedding = get_sentence_embedding(sentence)

    for pair in paired_data:
        retrieve_data = list(pair.keys())[0]
        datastore_embeddings.append(get_sentence_embedding(retrieve_data))

    # Compute cosine similarity between the query sentence and all sentences in the datastore
    similarities = cosine_similarity(query_embedding.reshape(1, -1), np.vstack(datastore_embeddings))

    top_k_indices = similarities[0].argsort()[-k:][::-1]
    for idx in top_k_indices:
        print(f"{ list(paired_data[int(idx)].keys())[0] } ----相似度{similarities[0][idx]}")
        retrieve_output.append([list(paired_data[int(idx)].keys())[0],list(paired_data[int(idx)].values())[0]])
    print("------------------------------------------------------")


query: keep theirs
keep his ----相似度0.9387737512588501
use ours ----相似度0.8443484902381897
let my ----相似度0.8390036225318909
get his ----相似度0.8353885412216187
get theirs ----相似度0.835381269454956
live her theirs ----相似度0.8325923681259155
use hers ----相似度0.8294834494590759
hear ours ----相似度0.8222799301147461
will his ----相似度0.8177357912063599
put hers ----相似度0.8175458908081055
------------------------------------------------------
query: come half case between he these world at enough Mr on a government
mean he under a mother between much right our work ----相似度0.8664097189903259
be either room since both country by those job in those part by a man between it either family ----相似度0.8609208464622498
give this thing between half case since his our world since you ----相似度0.8599960207939148
tell its business from each money of much Mr enough school between this story in every state on they ----相似度0.8519905209541321
come much room of she at some business in much word between its fact enough man s

In [ ]:
len(retrieve_output)

496

In [ ]:
# 將輸出保存為 JSON 檔案
output_json = json.dumps(retrieve_output, indent=2)
with open('/content/eng100_1_embedding61.json', 'w') as output_file:
    output_file.write(output_json)

In [ ]:
with open('eng100_1_embedding.json') as f:
    data = json.load(f)

In [ ]:
len(data)

1100

In [ ]:
len(retrieve_output)

188